In [16]:
import sys
sys.path.insert(0, "../../python")

import cProfile
from propagation import laser
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from line_profiler import LineProfiler

def do_profile(follow=[]):
    def inner(func):
        def profiled_func(*args, **kwargs):
            try:
                profiler = LineProfiler()
                profiler.add_function(func)
                for f in follow:
                    profiler.add_function(f)
                profiler.enable_by_count()
                return func(*args, **kwargs)
            finally:
                profiler.print_stats()
        return profiled_func
    return inner

In [17]:
Nx = 2**10
X = 100
x = np.linspace(-X/2, X/2, Nx, False)
Ny = 2**8
Y = 100
y = np.linspace(-Y/2, Y/2, Ny, False)

Nz = 2**8
Z = 100
z = np.linspace(0, Z, Nz)

w0 = 5
zR = np.pi * w0**2
wZ = w0 * np.sqrt(1 + (Z/zR)**2)
E = np.exp(-(np.reshape(x-10, (Nx, 1))**2 + np.reshape(y, (1, Ny))**2) / w0**2)
EE = (w0 / wZ) * np.exp(-(x-10)**2 / wZ**2)

cProfile.run('laser.fourier_prop2(E, x, y, z, 0.8)')

         121 function calls in 10.234 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.063    0.063   10.234   10.234 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:37(_any)
        2    0.000    0.000    0.011    0.005 fftpack.py:100(fft)
        2    0.824    0.412    4.280    2.140 fftpack.py:195(ifft)
        4    0.000    0.000    2.819    0.705 fftpack.py:46(_raw_fft)
        2    0.000    0.000    0.000    0.000 fftpack.py:598(_cook_nd_args)
        2    0.303    0.152    4.594    2.297 fftpack.py:617(_raw_fftnd)
        1    0.000    0.000    0.011    0.011 fftpack.py:819(fft2)
        1    0.000    0.000    4.583    4.583 fftpack.py:908(ifft2)
        4    0.000    0.000    0.000    0.000 fftpack.py:93(_unitary)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:127(reshape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1900(any)
        3    0.

In [18]:
@do_profile(follow=[laser.fourier_prop2])
def test(E, x, y, z):
    laser.fourier_prop2(E, x, y, z, 0.8)
    
result = test(E, x, y, z)
# I split the function up to see which part was slow, it ends up close to two thirds of the computation is spent
# calculating the np.exp(...) part, potentially can be spead up

Timer unit: 2.84444e-07 s

Total time: 10.6224 s
File: ../../python\propagation\laser.py
Function: fourier_prop2 at line 131

Line #      Hits         Time  Per Hit   % Time  Line Contents
   131                                           def fourier_prop2(E, x, y, z, lam, n=1):
   132                                               """ Propagates an electromagnetic wave from a 2D boundary.
   133                                           
   134                                               Uses the Rayleigh-Sommerfeld transfer function to propagate an
   135                                               electromagnetic wave from a 2D boundary. The calculation assumes a
   136                                               homogeneous index of refraction in the region of propagation.
   137                                           
   138                                               Parameters
   139                                               ----------
   140                        